# Custom Layers

### Layers without Parameters

To start, we construct a custom layer
that does not have any parameters of its own.
The following `CenteredLayer` class simply
subtracts the mean from its input.
To build it, we simply need to inherit
from the base layer class and implement the forward propagation function.


In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

Now incorporate our layer as a component in constructing more complex models.

As an extra sanity check, we can send random data through the network and check that the mean is in fact 0. Because we are dealing with floating point numbers, we may still see a very small nonzero number due to quantization.

In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(-1.1642e-08, grad_fn=<MeanBackward0>)

### Layers with Parameters

Now that we know how to define simple layers,
let's move on to defining layers with parameters
that can be adjusted through training.
We can use built-in functions to create parameters, which
provide some basic housekeeping functionality.
In particular, they govern access, initialization,
sharing, saving, and loading model parameters.
This way, among other benefits, we will not need to write
custom serialization routines for every custom layer.

Now let's implement our own version of the  fully connected layer.
Recall that this layer requires two parameters,
one to represent the weight and the other for the bias.
In this implementation, we bake in the ReLU activation as a default.
This layer requires two input arguments: `in_units` and `units`, which
denote the number of inputs and outputs, respectively.


In [4]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units): # in_units and units denote the number of inputs and outputs, respectively
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units)) ####
        self.bias = nn.Parameter(torch.randn(units))
    
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data ####
        return F.relu(linear)
    
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[ 0.6345,  0.6182,  0.9836],
        [-0.5523, -0.7240, -0.6292],
        [ 1.0063,  0.0704, -1.4594],
        [-0.3102, -0.6469, -0.8409],
        [ 2.4734, -0.7241,  0.2020]], requires_grad=True)

We can directly carry out forward propagation calculations using custom layers.

In [5]:
dense(torch.rand(2, 5))

tensor([[0.5111, 0.1303, 0.0000],
        [0.0000, 0.2674, 0.0000]])

We can also construct models using custom layers.

In [6]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.6392],
        [0.0000]])